In [1]:
import cmath
import math
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
def createFourierMatrix(k,n):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    i = cmath.sqrt(-1)
    val = cmath.exp(-2*cmath.pi*i/n)
    p = (k-1)/2
    q = (k+1)/2
    F = torch.zeros(n*n,k*k).to(device)
    F = F.type(torch.complex64)

    f = torch.zeros(n,1).to(device)
    f = f.type(torch.complex64)
    f_u = torch.zeros(n*n,1).to(device)
    f_u = f_u.type(torch.complex64)
    for u in range(n):
        f_u[u*n:(u+1)*n]=val**u
        f[u]=val**u;

    f_v = f.repeat(n,1);
    for u in range(k):
        for v in range(k):
            a=0
            b=0
            if(u<=p):
                a = n-p+u;
            else:
                a = u-p;


            if(v<=p):
                b = n-p+v;
            else:
                b = v-p;

            F[:,(u*k+v)]=((torch.pow(f_u,(a)))*(torch.pow(f_v,(b)))).flatten();

    return F

In [4]:
def zeroPad2DMatrix(layer_wt,n):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    k = layer_wt.size()[3]
    p = (k-1)/2
    q = (k+1)/2
    I = torch.eye(n).to(device);
    ind1 = torch.arange(0,p)
    ind2 = torch.arange(p,k)
    ind3 = torch.arange(k,n)
    indices = torch.cat((ind2,ind3,ind1))
    indices=indices.type(torch.int64)
    perm = I[indices].to(device)
    perm_mat = perm.unsqueeze(0).unsqueeze(0)
    pad_left = 0
    pad_right = n - k
    pad_top = 0
    pad_bottom = n - k
    # Apply padding
    padded_wt = torch.nn.functional.pad(layer_wt, (pad_left, pad_right, pad_top, pad_bottom)).to(device)
    perm_mat_tr = torch.transpose(perm_mat,2,3)
    padded_final = torch.matmul(perm_mat,torch.matmul(padded_wt,perm_mat_tr))
    return padded_final

In [5]:
def deZeroPad2DMatrix(layer_wt,k):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    n = layer_wt.size()[-1]
    p = (k-1)/2
    q = (k+1)/2
    I = torch.eye(n).to(device);
    ind1 = torch.arange(0,p)
    ind2 = torch.arange(p,k)
    ind3 = torch.arange(k,n)
    indices = torch.cat((ind2,ind3,ind1))
    indices=indices.type(torch.int64)
    perm = I[indices].to(device)
    #perm_mat = perm.unsqueeze(0).unsqueeze(0)

    # Apply padding
    perm_mat_tr = torch.transpose(perm,-2,-1)

    unpadded_wt = torch.matmul(perm_mat_tr,torch.matmul(layer_wt,perm))

    layer = unpadded_wt[:,:,:k,:k].to(device)
    return layer

In [6]:
def computeLayerLipschitzFourier(layer_wt,n):
    layer_wt_padded = zeroPad2DMatrix(layer_wt,n)
    layer_pf=torch.fft.fft2(layer_wt_padded)
    layer_fperm = torch.permute(layer_pf,(2,3,0,1))
    sing = torch.linalg.svdvals(layer_fperm)
    lip = torch.max(torch.abs(sing))
    return lip

In [7]:
import torchvision.models as models
alexnet_model = models.alexnet(pretrained=True)

/home/kunallab/anaconda3/envs/sayan/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/kunallab/anaconda3/envs/sayan/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
layer = alexnet_model.features[8].weight.to(device)
lip = computeLayerLipschitzFourier(layer,13)
print(lip)

tensor(14.6738, device='cuda:0', grad_fn=<MaxBackward1>)


In [8]:
def createComplexVerticesL2Norm(dim):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    vertices = torch.empty((dim, 0), dtype=torch.complex64).to(device)
    I_dim = torch.eye(dim, dtype=torch.complex64).to(device)
    vertices = torch.cat((I_dim, -I_dim), dim=1)
    return vertices

In [ ]:
vertices = createComplexVerticesL2Norm(layer.shape[0]*layer.shape[1])
print(vertices.shape)

In [12]:
s = layer.shape
k = s[3]
n = 13

F = createFourierMatrix(k,n)

In [22]:
#Polytope Projection
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Constants (example values, replace these with actual data)
H0 = torch.reshape(layer,(s[0]*s[1],s[2]*s[3])).to(device)
V = vertices.to(device)
F = F.to(device)
V_real, V_imag = torch.real(V), torch.imag(V)
F_real, F_imag = torch.real(F), torch.imag(F)

print(F_real.dtype)
# Initialize optimization variables (H and lambda)
H = torch.rand_like(H0, requires_grad=True,dtype = torch.float32).to(device)
lambda_ = torch.rand((V.shape[1], n*n), requires_grad=True,dtype = torch.float32,device=device)  # size based on V and H0
# Optimizer
optimizer = torch.optim.Adam([H, lambda_], lr=0.01)

# Penalty weights
penalty_weight_eq = 1000.0
penalty_weight_ineq = 1000.0

# Optimization loop
for step in range(1000):
    optimizer.zero_grad()  # Clear previous gradients

    # Objective function: Frobenius norm of (H - H0)
    loss = torch.norm(H - H0, p='fro')

    # Equality constraint: V * lambda = H * F
    eq_constraint_real = V_real @ lambda_ - H@F_real.T
    eq_constraint_imag = V_imag @ lambda_ - H@F_imag.T
    eq_constraint = torch.sqrt(eq_constraint_real**2 + eq_constraint_imag**2)
    loss = loss+ penalty_weight_eq * torch.norm(eq_constraint,p='fro')  # Penalty for the equality constraint

    # Column sum constraint: sum of each column of lambda = 1
    col_sum_constraint = torch.abs(torch.sum(lambda_, dim=0) - 1)
    loss = loss + penalty_weight_eq * torch.sum(col_sum_constraint)  # Penalty for the column sum constraint

    # Inequality constraint: lambda >= 0
    ineq_constraint = torch.norm(torch.nn.functional.relu(lambda_) - lambda_,p = 'fro')
    loss = loss + penalty_weight_eq * torch.sum(ineq_constraint)

    # Backward pass and optimization step
    torch.autograd.set_detect_anomaly(True)
    loss.backward()
    optimizer.step()

    # Print progress
    if step % 100 == 0:
        print(f'Step {step}, loss = {loss.item()}')

# Results
print(f'Optimized H:\n{H}')
print(f'Optimized lambda:\n{lambda_}')

torch.float32
Step 0, loss = 19673526272.0
Step 100, loss = 82443960.0
Step 200, loss = 41432460.0
Step 300, loss = 41801620.0
Step 400, loss = 36457856.0
Step 500, loss = 43181500.0
Step 600, loss = 39966356.0
Step 700, loss = 41805592.0
Step 800, loss = 40656724.0
Step 900, loss = 41811604.0
Optimized H:
tensor([[ 0.0106,  0.0100, -0.0029,  ..., -0.0029,  0.0100,  0.0106],
        [-0.0083, -0.0044,  0.0015,  ...,  0.0015, -0.0044, -0.0083],
        [-0.0037,  0.0020,  0.0030,  ...,  0.0030,  0.0020, -0.0037],
        ...,
        [-0.0052, -0.0023, -0.0026,  ..., -0.0026, -0.0023, -0.0052],
        [ 0.0022,  0.0055, -0.0002,  ..., -0.0002,  0.0055,  0.0022],
        [-0.0126,  0.0063,  0.0002,  ...,  0.0002,  0.0063, -0.0126]],
       device='cuda:0', requires_grad=True)
Optimized lambda:
tensor([[ 0.0796,  0.2529, -0.3796,  ..., -0.1417, -0.3499, -0.4002],
        [-0.3885, -0.3099, -0.1370,  ...,  0.3589, -0.0375,  0.2595],
        [-0.2342, -0.4731, -0.0374,  ...,  0.1264,  0.20

In [23]:
torch.sum(torch.abs(torch.sum(lambda_, dim=0) - 1))

tensor(40949.4922, device='cuda:0', grad_fn=<SumBackward0>)

In [19]:
loss = torch.norm(H - H0, p='fro')
print(loss)

tensor(26.4691, grad_fn=<LinalgVectorNormBackward0>)


In [25]:
layer_wt = torch.reshape(H,(s[0],s[1],s[2],s[3]))
layer_wt = layer_wt.to('cpu')
print(computeLayerLipschitzFourier(layer_wt,40))

tensor(1.1367, grad_fn=<MaxBackward1>)


In [27]:
#Frobenius Norm Projection - Constraint with RELU
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

layer = alexnet_model.features[3].weight.to(device)
s = layer.shape
k = s[3]
n = 13

F = createFourierMatrix(k,n)

# Constants (example values, replace these with actual data)
all_one = torch.ones(n*n,device=device)
H0 = torch.reshape(layer,(s[0]*s[1],s[2]*s[3])).to(device)
F = F.to(device)
F_real, F_imag = torch.real(F), torch.imag(F)

# Initialize optimization variables (H and lambda)
H = torch.rand_like(H0, requires_grad=True,dtype = torch.float32).to(device)

# Optimizer
optimizer = torch.optim.Adam([H], lr=0.01)

# Penalty weights
penalty_weight_eq = 1000.0
penalty_weight_ineq = 1000.0

# Optimization loop
for step in range(1000):
    optimizer.zero_grad()  # Clear previous gradients

    # Objective function: Frobenius norm of (H - H0)
    loss = torch.norm(H - H0, p='fro')

    # Equality constraint: V * lambda = H * F
    ineq_constraint_real = H@F_real.T
    ineq_constraint_imag = H@F_imag.T
    ineq_constraint_sum = 1 - torch.sum((ineq_constraint_real**2 + ineq_constraint_imag**2),dim=0)

    # Inequality constraint: lambda >= 0
    ineq_constraint = torch.norm(torch.nn.functional.relu(ineq_constraint_sum) - ineq_constraint_sum,p = 'fro')
    loss = loss + penalty_weight_ineq * torch.sum(ineq_constraint)

    # Backward pass and optimization step
    torch.autograd.set_detect_anomaly(True)
    loss.backward()
    optimizer.step()

    # Print progress
    if step % 100 == 0:
        print(f'Step {step}, loss = {loss.item()}')

# Results
print(f'Optimized H:\n{H}')

Step 0, loss = 3501525760.0
Step 100, loss = 90510584.0
Step 200, loss = 2879210.75
Step 300, loss = 30224.744140625
Step 400, loss = 26.170150756835938
Step 500, loss = 26.17002296447754
Step 600, loss = 26.169885635375977
Step 700, loss = 26.16973304748535
Step 800, loss = 26.16956329345703
Step 900, loss = 26.169376373291016
Optimized H:
tensor([[-2.5685e-05, -3.4364e-05, -6.8903e-05,  ..., -4.7072e-05,
         -2.3813e-05, -1.6826e-05],
        [-2.9484e-06, -4.2947e-07, -1.2389e-06,  ..., -1.1497e-06,
         -2.2666e-06,  7.3690e-07],
        [-2.7154e-04, -6.3394e-04, -9.6890e-04,  ...,  2.4672e-03,
         -6.1208e-04, -3.6221e-04],
        ...,
        [-3.3118e-05, -3.0584e-05, -7.6941e-06,  ...,  5.3623e-05,
         -5.2775e-05, -5.9902e-05],
        [-1.1631e-04, -1.3938e-04, -1.0898e-04,  ..., -8.2066e-05,
         -5.7592e-05, -2.0274e-05],
        [ 6.9687e-06,  1.6077e-06, -3.3218e-07,  ..., -8.5423e-07,
         -5.2531e-06,  2.4060e-06]], device='cuda:0', requires

In [28]:
#Output Test
layer_wt = torch.reshape(H,(s[0],s[1],s[2],s[3]))
print(f'Lipschitz Constant: {computeLayerLipschitzFourier(layer_wt,n)}')
print(f'Difference:{torch.norm(H - H0, p='fro')}')
print(f'Original H:\n{H0}')

Lipschitz Constant: 0.384726345539093
Difference:26.16917610168457
Original H:
tensor([[ 0.0036,  0.0014,  0.0372,  ...,  0.0315,  0.0167, -0.0047],
        [-0.0055, -0.0191, -0.0324,  ..., -0.0022, -0.0166,  0.0056],
        [-0.0126, -0.0594, -0.0963,  ..., -0.0080, -0.0818,  0.0144],
        ...,
        [-0.0065, -0.0329, -0.0157,  ...,  0.0004,  0.0075,  0.0109],
        [ 0.0207,  0.0207,  0.0766,  ..., -0.0105, -0.0137, -0.0025],
        [ 0.0704,  0.0215, -0.0091,  ..., -0.0205,  0.0170,  0.0243]],
       device='cuda:0', grad_fn=<ViewBackward0>)


In [24]:
layer = alexnet_model.features[3].weight.to(device)
s = layer.shape
k = s[3]
n = 13

F = createFourierMatrix(k,n)

In [25]:
# Bhartendu with loop - FAILED
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Constants (example values, replace these with actual data)
all_one = torch.ones(n*n,device=device)
H0 = torch.reshape(layer,(s[0]*s[1],s[2]*s[3])).to(device)
F = F.to(device)
F_real, F_imag = torch.real(F), torch.imag(F)

# Initialize optimization variables (H and lambda)
H = torch.rand_like(H0, requires_grad=True,dtype = torch.float32).to(device)

# Optimizer
optimizer = torch.optim.Adam([H], lr=0.01)

# Penalty weights
penalty_weight_eq = 1000.0
penalty_weight_ineq = 1000.0

# Optimization loop
for step in range(1000):
    optimizer.zero_grad()  # Clear previous gradients

    # Objective function: Frobenius norm of (H - H0)
    loss = torch.norm(H - H0, p='fro')

    # Equality constraint: V * lambda = H * F
    Hf_real = H@F_real.T
    Hf_imag = H@F_imag.T
    Hf = torch.sqrt(Hf_real**2 + Hf_imag**2)
    for i in range(n*n):
        mat = torch.reshape(Hf[:,i],(s[0],s[1]))
        sum_col = 1-torch.sum(mat,dim=0)
        sum_row = 1-torch.sum(mat,dim=1)
        loss = loss +penalty_weight_ineq* torch.norm(torch.nn.functional.relu(sum_col) - sum_col,p = 'fro')
        loss = loss + penalty_weight_ineq*torch.norm(torch.nn.functional.relu(sum_row) - sum_row,p = 'fro')

    # Backward pass and optimization step
    torch.autograd.set_detect_anomaly(True)
    loss.backward()
    optimizer.step()

    # Print progress
    if step % 100 == 0:
        print(f'Step {step}, loss = {loss.item()}')

# Results
print(f'Optimized H:\n{H}')

Step 0, loss = 852986304.0


In [25]:
#Frobenius Norm Projection - Constraint with abs
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

layer = alexnet_model.features[3].weight.to(device)
s = layer.shape
k = s[3]
n = 13

F = createFourierMatrix(k,n)

# Constants (example values, replace these with actual data)
all_one = torch.ones(n*n,device=device)
H0 = torch.reshape(layer,(s[0]*s[1],s[2]*s[3])).to(device)
F = F.to(device)
F_real, F_imag = torch.real(F), torch.imag(F)

# Initialize optimization variables (H and lambda)
H = torch.rand_like(H0, requires_grad=True,dtype = torch.float32).to(device)

# Optimizer
optimizer = torch.optim.Adam([H], lr=0.01)

# Penalty weights
penalty_weight_eq = 1000.0
penalty_weight_ineq = 10000.0

# Optimization loop
for step in range(1000):
    optimizer.zero_grad()  # Clear previous gradients

    # Objective function: Frobenius norm of (H - H0)
    loss = torch.norm(H - H0, p='fro')

    # Equality constraint: V * lambda = H * F
    ineq_constraint_real = H@F_real.T
    ineq_constraint_imag = H@F_imag.T
    ineq_constraint_sum = 1 - torch.sum((ineq_constraint_real**2 + ineq_constraint_imag**2),dim=0)

    # Inequality constraint: lambda >= 0
    ineq_constraint = torch.norm(torch.abs(ineq_constraint_sum) - ineq_constraint_sum,p = 'fro')
    loss = loss + penalty_weight_ineq * torch.sum(ineq_constraint)

    # Backward pass and optimization step
    torch.autograd.set_detect_anomaly(True)
    loss.backward()
    optimizer.step()

    # Print progress
    if step % 100 == 0:
        print(f'Step {step}, loss = {loss.item()}')

# Results
print(f'Optimized H:\n{H}')

Step 0, loss = 70200131584.0
Step 100, loss = 1816661632.0
Step 200, loss = 57875504.0
Step 300, loss = 605619.5
Step 400, loss = 26.17026138305664
Step 500, loss = 26.170249938964844
Step 600, loss = 26.170242309570312
Step 700, loss = 26.17023468017578
Step 800, loss = 26.170228958129883
Step 900, loss = 26.17021942138672
Optimized H:
tensor([[-2.9761e-05, -4.5923e-05, -1.1979e-04,  ..., -2.6065e-04,
          7.2066e-04, -2.4547e-05],
        [-1.0144e-04,  7.1570e-04, -1.6990e-04,  ...,  1.1148e-03,
          1.2668e-03,  1.3764e-04],
        [-1.4357e-04, -2.8224e-04, -1.1838e-04,  ..., -5.9225e-04,
         -4.2574e-04, -2.1453e-04],
        ...,
        [-3.5816e-05, -5.8492e-05, -3.4246e-05,  ..., -1.1027e-04,
          1.9379e-04,  2.9805e-05],
        [-4.5587e-04,  1.3412e-03,  3.9402e-03,  ..., -1.2584e-03,
         -9.3011e-04, -6.2155e-04],
        [-7.8389e-05, -1.0949e-04, -7.2908e-05,  ..., -2.7899e-05,
         -1.1135e-05, -2.1568e-06]], device='cuda:0', requires_gra

In [26]:
#Output Test
layer_wt = torch.reshape(H,(s[0],s[1],s[2],s[3]))
print(f'Lipschitz Constant: {computeLayerLipschitzFourier(layer_wt,n)}')
print(f'Difference:{torch.norm(H - H0, p='fro')}')
print(f'Original H:\n{H0}')


Lipschitz Constant: 0.3922692835330963
Difference:26.170207977294922
Original H:
tensor([[ 0.0036,  0.0014,  0.0372,  ...,  0.0315,  0.0167, -0.0047],
        [-0.0055, -0.0191, -0.0324,  ..., -0.0022, -0.0166,  0.0056],
        [-0.0126, -0.0594, -0.0963,  ..., -0.0080, -0.0818,  0.0144],
        ...,
        [-0.0065, -0.0329, -0.0157,  ...,  0.0004,  0.0075,  0.0109],
        [ 0.0207,  0.0207,  0.0766,  ..., -0.0105, -0.0137, -0.0025],
        [ 0.0704,  0.0215, -0.0091,  ..., -0.0205,  0.0170,  0.0243]],
       device='cuda:0', grad_fn=<ViewBackward0>)


In [21]:
#Bhartendu Constraint - Projected on Constraints
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Constants (example values, replace these with actual data)
all_one = torch.ones(n*n,device=device)
H0 = torch.reshape(layer,(s[0]*s[1],s[2]*s[3])).to(device)
F = F.to(device)
F_real, F_imag = torch.real(F), torch.imag(F)

# Initialize optimization variables (H and lambda)
H = torch.rand_like(H0, requires_grad=True,dtype = torch.float32).to(device)

# Optimizer
optimizer = torch.optim.Adam([H], lr=0.01)

# Penalty weights
penalty_weight_eq = 1000.0
penalty_weight_ineq = 1000.0

# Optimization loop
for step in range(1000):
    optimizer.zero_grad()  # Clear previous gradients

    # Objective function: Frobenius norm of (H - H0)
    loss = torch.norm(H - H0, p='fro')

    # Equality constraint: V * lambda = H * F
    Hf_real = H@F_real.T
    Hf_imag = H@F_imag.T
    Hf = torch.sqrt(Hf_real**2 + Hf_imag**2)
    H_mat = torch.permute(torch.reshape(Hf,(s[0],s[1],n,n)),(2,3,0,1))

    H1_norm = torch.clamp(torch.sum(H_mat, dim=2),min=1.0)
    s_h1 = H1_norm.shape;
    H_mat_1_norm = torch.div(H_mat,torch.reshape(H1_norm,(s_h1[0],s_h1[1],1,s_h1[2])))
    Hinf_norm = torch.clamp(torch.sum(H_mat_1_norm, dim=3),min=1.0)
    s_hinf = Hinf_norm.shape;
    H_mat_norm = torch.div(H_mat_1_norm,torch.reshape(Hinf_norm,(s_hinf[0],s_hinf[1],s_hinf[2],1)))
    
    loss = loss + penalty_weight_ineq*(torch.norm((H_mat-H_mat_norm),p='fro'))


    # Backward pass and optimization step
    torch.autograd.set_detect_anomaly(True)
    loss.backward()
    optimizer.step()

    # Print progress
    if step % 100 == 0:
        print(f'Step {step}, loss = {loss.item()}')

# Results
print(f'Optimized H:\n{H}')

Step 0, loss = 4154583.25
Step 100, loss = 43754.02734375
Step 200, loss = 1723.7564697265625
Step 300, loss = 1105.1204833984375
Step 400, loss = 671.1167602539062
Step 500, loss = 508.1769714355469
Step 600, loss = 366.5533142089844
Step 700, loss = 914.8499145507812
Step 800, loss = 661.396728515625
Step 900, loss = 1072.9964599609375
Optimized H:
tensor([[ 5.9885e-04, -6.3687e-04,  1.4169e-03,  ...,  1.4092e-03,
          2.9666e-04, -7.8557e-04],
        [ 5.8077e-04, -8.6543e-04,  8.2217e-05,  ...,  2.3715e-04,
          1.7002e-04, -1.8742e-04],
        [-8.1851e-05, -6.9085e-04, -1.1079e-04,  ...,  3.7614e-05,
         -4.7068e-05,  5.5482e-04],
        ...,
        [-9.3642e-05, -1.1380e-03,  1.1766e-03,  ..., -8.5665e-05,
         -1.4870e-04,  2.2091e-04],
        [-3.1953e-04,  7.8229e-04,  2.5259e-03,  ...,  7.2739e-05,
         -1.2424e-05, -3.6301e-04],
        [ 1.2187e-03,  1.7081e-04,  3.1731e-04,  ...,  1.0625e-03,
          7.7116e-05,  2.7824e-04]], device='cuda:0'

In [33]:
#Output Test
layer_wt = torch.reshape(H,(s[0],s[1],s[2],s[3]))
print(f'Lipschitz Constant: {computeLayerLipschitzFourier(layer_wt,n)}')
print(f'Difference:{torch.norm(H - H0, p='fro')}')
print(f'Original H:\n{H0}')


Lipschitz Constant: 2.309136390686035
Difference:26.436567306518555
Original H:
tensor([[ 0.0036,  0.0014,  0.0372,  ...,  0.0315,  0.0167, -0.0047],
        [-0.0055, -0.0191, -0.0324,  ..., -0.0022, -0.0166,  0.0056],
        [-0.0126, -0.0594, -0.0963,  ..., -0.0080, -0.0818,  0.0144],
        ...,
        [-0.0065, -0.0329, -0.0157,  ...,  0.0004,  0.0075,  0.0109],
        [ 0.0207,  0.0207,  0.0766,  ..., -0.0105, -0.0137, -0.0025],
        [ 0.0704,  0.0215, -0.0091,  ..., -0.0205,  0.0170,  0.0243]],
       device='cuda:0', grad_fn=<ViewBackward0>)


In [ ]:
#Bhartendu Constraint No Loop - FAILED
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Constants (example values, replace these with actual data)
all_one = torch.ones(n*n,device=device)
H0 = torch.reshape(layer,(s[0]*s[1],s[2]*s[3])).to(device)
F = F.to(device)
F_real, F_imag = torch.real(F), torch.imag(F)

# Initialize optimization variables (H and lambda)
H = torch.rand_like(H0, requires_grad=True,dtype = torch.float32).to(device)

# Optimizer
optimizer = torch.optim.Adam([H], lr=0.01)

# Penalty weights
penalty_weight_eq = 1000.0
penalty_weight_ineq = 10000.0

# Optimization loop
for step in range(4000):
    optimizer.zero_grad()  # Clear previous gradients

    # Objective function: Frobenius norm of (H - H0)
    loss = torch.norm(H - H0, p='fro')*0.1
    loss.backward()
    optimizer.step()

    # Backward pass and optimization step
    torch.autograd.set_detect_anomaly(True)
    
    # Print progress
    if step % 100 == 0:
        print(f'Step {step}, loss = {loss.item()}')

    # Equality constraint: V * lambda = H * F
    Hf_real = H@F_real.T
    Hf_imag = H@F_imag.T
    Hf = torch.sqrt(Hf_real**2 + Hf_imag**2)
    H_mat = torch.permute(torch.reshape(Hf,(s[0],s[1],n,n)),(2,3,0,1))
    
    H1_norm = 1-torch.max(torch.sum(H_mat, dim=2))
    Hinf_norm = 1-torch.max(torch.sum(H_mat, dim=3))
    loss = loss + penalty_weight_ineq*(torch.norm((torch.abs(H1_norm)-H1_norm),p='fro'))
    loss = loss + penalty_weight_ineq*(torch.norm((torch.abs(Hinf_norm)-Hinf_norm),p='fro'))

    

# Results
print(f'Optimized H:\n{H}')

In [16]:
r = torch.rand(2,3,4)
print(r)

tensor([[[0.9720, 0.5755, 0.5245, 0.9264],
         [0.3239, 0.5982, 0.4861, 0.7159],
         [0.8257, 0.7799, 0.8597, 0.6664]],

        [[0.7571, 0.6749, 0.7261, 0.8427],
         [0.8832, 0.8098, 0.9694, 0.1520],
         [0.4983, 0.1817, 0.6956, 0.5596]]])


In [12]:
temp = torch.sum(r,dim=1)
print(temp)
temp = torch.reshape(temp,(2,1,4))
torch.div(r,temp)

tensor([[1.6868, 0.9763, 1.7824, 1.3842],
        [1.9146, 2.0612, 0.8041, 1.5873]])


tensor([[[0.3349, 0.3453, 0.4519, 0.5676],
         [0.0794, 0.0190, 0.3525, 0.0844],
         [0.5857, 0.6356, 0.1956, 0.3481]],

        [[0.3671, 0.3947, 0.0061, 0.5868],
         [0.4346, 0.2387, 0.3102, 0.0871],
         [0.1983, 0.3666, 0.6837, 0.3261]]])

In [18]:
temp = torch.sum(r,dim=2)
print(temp)
temp = torch.reshape(temp,(2,3,1))
torch.div(r,temp)

tensor([[2.9983, 2.1242, 3.1318],
        [3.0008, 2.8145, 1.9352]])


tensor([[[0.3242, 0.1919, 0.1749, 0.3090],
         [0.1525, 0.2816, 0.2288, 0.3370],
         [0.2637, 0.2490, 0.2745, 0.2128]],

        [[0.2523, 0.2249, 0.2420, 0.2808],
         [0.3138, 0.2877, 0.3444, 0.0540],
         [0.2575, 0.0939, 0.3594, 0.2892]]])

In [15]:
temp = torch.ones(2)
temp2 = torch.zeros(2,dtype=torch.complex64)
temp2.imag = temp
print(temp2)

tensor([0.+1.j, 0.+1.j])


In [80]:
layer = torch.tensor([[[[0,0.3,0],[0,0.4,0],[0,0.3,0]]]])
print(layer)

tensor([[[[0.0000, 0.3000, 0.0000],
          [0.0000, 0.4000, 0.0000],
          [0.0000, 0.3000, 0.0000]]]])


In [72]:
layer.shape
computeLayerLipschitzFourier(layer,13)

tensor(1., device='cuda:0')

In [8]:
def SGDminimize(X0,Z,U,F,rho):
    F_real, F_imag = torch.real(F), torch.imag(F)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    X = torch.rand_like(X0, requires_grad=True,dtype = torch.float32).to(device)
    optimizer = torch.optim.Adam([X], lr=0.01)
    for step in range(1000):
        optimizer.zero_grad()
        loss = torch.norm(X - X0, p='fro')**2
        res_real = X@F_real.T - Z.real + U.real
        res_imag = X@F_imag.T - Z.imag + U.imag 
        loss = loss + (rho/2)*torch.sum((res_imag**2 + res_real**2))

        loss.backward()
        optimizer.step()
        #if(step%100 ==0):
        #    print(loss)
    
    return X.detach()

In [32]:
#ADMM Frobenius Normalize
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#layer = alexnet_model.features[3].weight.to(device)
layer = layer.to(device)
s = layer.shape
k = s[3]
n = 13

F = createFourierMatrix(k,n)

# Constants (example values, replace these with actual data)
all_one = torch.ones(n*n,device=device)
H0 = torch.reshape(layer,(s[0]*s[1],s[2]*s[3])).to(device)
print(H0.shape)
F = F.to(device)
print(F.shape)
F_real, F_imag = torch.real(F), torch.imag(F)

# Initialize optimization variables (H and lambda)
H = torch.rand_like(H0,dtype = torch.float32).to(device)
print(H.shape)
U = torch.zeros((s[0]*s[1],n*n),dtype=torch.complex64).to(device)
Hf = torch.zeros((s[0]*s[1],n*n),dtype=torch.complex64).to(device)

pri_res = 100
rho = 0.1

#i = 0
#optimizer = torch.optim.Adam([H], lr=0.01)
#while pri_res>0.5:
for i in range(100):
    # Optimize H
    H = SGDminimize(H0,Hf,U,F,rho)

    H_fourier = torch.zeros(Hf.shape,dtype = torch.complex64).to(device)
    H_fourier.real = H@torch.real(F.T)
    H_fourier.imag = H@torch.imag(F.T)
    #print(H_fourier)
    #Optimize Hf

    Hf = H_fourier + U
    H_frob = torch.clamp(torch.sqrt(torch.sum(torch.square(torch.abs(Hf)),dim = 0)),min=1)
    #H_frob = torch.sum(torch.abs(Hf),dim = 0)
    #print(H_frob)
    s_f = H_frob.shape
    Hf = torch.div(Hf,torch.reshape(H_frob,(1,s_f[0])))

    # Update U
    U = U + H_fourier - Hf 
    pri_res = torch.norm(H_fourier - Hf, p='fro')
    #i=i+1
    if(i%10 == 0):
        print(f'Step {i}: Primal Residual:{pri_res}')
# Results
print(f'Optimized H:\n{H}')

torch.Size([36, 9])
torch.Size([169, 9])
torch.Size([36, 9])
Step 0: Primal Residual:6.122568130493164
Step 10: Primal Residual:0.3624890446662903
Step 20: Primal Residual:0.21123899519443512
Step 30: Primal Residual:0.13808706402778625
Step 40: Primal Residual:0.0900331363081932
Step 50: Primal Residual:0.05904947593808174
Step 60: Primal Residual:0.0378074124455452
Step 70: Primal Residual:0.025564517825841904
Step 80: Primal Residual:0.017265474423766136
Step 90: Primal Residual:0.01165198627859354
Optimized H:
tensor([[-7.1758e-02,  3.2251e-02,  9.8408e-02,  1.0451e-02, -2.7271e-02,
          5.6846e-02, -4.0494e-02, -2.4730e-02,  1.1890e-01],
        [-1.2736e-02, -2.5142e-02,  6.5172e-02,  6.3220e-02, -8.2236e-02,
         -5.6564e-03,  9.6499e-02,  5.6072e-02,  5.2354e-02],
        [ 8.6255e-02, -5.5742e-02, -1.6814e-02, -3.3595e-02, -3.7666e-02,
         -5.2177e-02, -2.5699e-03,  1.2814e-01,  1.1503e-01],
        [ 2.6714e-02, -9.6533e-03,  8.3881e-02,  5.8944e-02, -4.7699e-02

In [10]:
#Output Test
layer_wt = torch.reshape(H,(s[0],s[1],s[2],s[3]))
print(f'Lipschitz Constant: {computeLayerLipschitzFourier(layer_wt,n)}')
print(f'Difference:{torch.norm(H - H0, p='fro')}')
print(f'Original H:\n{H0}')

Lipschitz Constant: 0.007294982671737671
Difference:26.133201599121094
Original H:
tensor([[ 0.0036,  0.0014,  0.0372,  ...,  0.0315,  0.0167, -0.0047],
        [-0.0055, -0.0191, -0.0324,  ..., -0.0022, -0.0166,  0.0056],
        [-0.0126, -0.0594, -0.0963,  ..., -0.0080, -0.0818,  0.0144],
        ...,
        [-0.0065, -0.0329, -0.0157,  ...,  0.0004,  0.0075,  0.0109],
        [ 0.0207,  0.0207,  0.0766,  ..., -0.0105, -0.0137, -0.0025],
        [ 0.0704,  0.0215, -0.0091,  ..., -0.0205,  0.0170,  0.0243]],
       device='cuda:0', grad_fn=<ViewBackward0>)


In [49]:
def SGDMinProject(H0,Ybar,Hbar,rho):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    H = torch.rand_like(H0, requires_grad=True,dtype = torch.complex64).to(device)
    optimizer = torch.optim.Adam([H], lr=0.01)
    for step in range(500):
        optimizer.zero_grad()
        loss = torch.norm(H - H0, p='fro')**2
        loss = loss + (rho) * (torch.norm(H-Hbar-Ybar,p='fro')**2)

        loss.backward()
        optimizer.step()
        #if(step%100 ==0):
        #    print(loss)
    
    return H.detach()

In [50]:
#ADMM Frobenius Normalize
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

layer = alexnet_model.features[3].weight.to(device)
#layer = layer.to(device)
s = layer.shape
k = s[3]
n = 13

F = createFourierMatrix(k,n)

# Constants (example values, replace these with actual data)
all_one = torch.ones(n*n,device=device)
H0 = torch.reshape(layer,(s[0]*s[1],s[2]*s[3])).to(device)
print(H0.shape)
F = F.to(device)
print(F.shape)
F_real, F_imag = torch.real(F), torch.imag(F)

# Initialize optimization variables (H and lambda)
H = torch.rand_like(H0,dtype = torch.float32).to(device)
print(H.shape)
U = torch.zeros((s[0]*s[1],n*n),dtype=torch.complex64).to(device)
Hf = torch.zeros((s[0]*s[1],n*n),dtype=torch.complex64).to(device)

pri_res = 100
rho = 0.1

#i = 0
#optimizer = torch.optim.Adam([H], lr=0.01)
#while pri_res>0.5:
for i in range(100):
    # Optimize H
    H = SGDminimize(H0,Hf,U,F,rho)

    H_fourier = torch.zeros(Hf.shape,dtype = torch.complex64).to(device)
    H_fourier.real = H@torch.real(F.T)
    H_fourier.imag = H@torch.imag(F.T)
    #print(H_fourier)
    #Optimize Hf

    Hf_0 = H_fourier + U
    Y1 = torch.zeros(Hf_0.shape,dtype=torch.complex64,device=device)
    Y2 = torch.zeros(Hf_0.shape,dtype=torch.complex64,device=device)
    
    Hf_1 = Hf_0
    Hf_2 = Hf_0
    Hf_z = Hf_0
    H_bar = (Hf_1+Hf_2)/2
    Y_bar = (Y1 + Y2)/2

    for j in range(100):
        X1 = Hf_z - Y1
        X1_mat = torch.permute(torch.reshape(X1,(s[0],s[1],n,n)),(2,3,0,1))
        X1_norm = torch.clamp(torch.sum(torch.abs(X1_mat), dim=2),min=1.0)
        Hf_1 = torch.div(X1_mat,torch.reshape(X1_norm,((n,n,1,s[1]))))
        Hf_1 = torch.reshape(torch.permute(Hf_1,(2,3,0,1)),(s[0]*s[1],n*n))

        X1 = Hf_z - Y2
        X1_mat = torch.permute(torch.reshape(X1,(s[0],s[1],n,n)),(2,3,0,1))
        X1_norm = torch.clamp(torch.sum(torch.abs(X1_mat), dim=3),min=1.0)
        Hf_2 = torch.div(X1_mat,torch.reshape(X1_norm,((n,n,s[0],1))))
        Hf_2 = torch.reshape(torch.permute(Hf_2,(2,3,0,1)),(s[0]*s[1],n*n))

        H_bar = (Hf_1+Hf_2)/2
        Hf_z = SGDMinProject(Hf_0,Y_bar,H_bar,0.01)
        
        Y1 = Y1 + (Hf_1 - Hf_z)
        Y2 = Y2 + (Hf_2 - Hf_z)
        Y_bar = (Y1 + Y2)/2
        if(j%10 == 0):
            print(f'----------------Step {j}: Primal Residual:{torch.norm(Hf_z - H_bar,p='fro')}')
        
    Hf = Hf_z

    # Update U
    U = U + H_fourier - Hf 
    pri_res = torch.norm(H_fourier - Hf, p='fro')
    #i=i+1
    if(i%10 == 0):
        print(f'Step {i}: Primal Residual:{pri_res}')
# Results
print(f'Optimized H:\n{H}')

torch.Size([12288, 25])
torch.Size([169, 25])
torch.Size([12288, 25])
----------------Step 0: Primal Residual:21.07489013671875
----------------Step 10: Primal Residual:19.143451690673828
----------------Step 20: Primal Residual:17.391773223876953
----------------Step 30: Primal Residual:15.80850601196289
----------------Step 40: Primal Residual:14.377713203430176
----------------Step 50: Primal Residual:13.086310386657715
----------------Step 60: Primal Residual:11.919387817382812
----------------Step 70: Primal Residual:10.866209030151367
----------------Step 80: Primal Residual:9.915656089782715
----------------Step 90: Primal Residual:9.0564603805542
Step 0: Primal Residual:13.667876243591309
----------------Step 0: Primal Residual:41.54755401611328
----------------Step 10: Primal Residual:37.64556121826172
----------------Step 20: Primal Residual:34.11109161376953
----------------Step 30: Primal Residual:30.913127899169922
----------------Step 40: Primal Residual:28.02005767822265

KeyboardInterrupt: 

In [51]:
#Output Test
layer_wt = torch.reshape(H,(s[0],s[1],s[2],s[3]))
print(f'Lipschitz Constant: {computeLayerLipschitzFourier(layer_wt,n)}')
print(f'Difference:{torch.norm(H - H0, p='fro')}')
print(f'Original H:\n{H0}')

Lipschitz Constant: 1.437525987625122
Difference:23.297317504882812
Original H:
tensor([[ 0.0036,  0.0014,  0.0372,  ...,  0.0315,  0.0167, -0.0047],
        [-0.0055, -0.0191, -0.0324,  ..., -0.0022, -0.0166,  0.0056],
        [-0.0126, -0.0594, -0.0963,  ..., -0.0080, -0.0818,  0.0144],
        ...,
        [-0.0065, -0.0329, -0.0157,  ...,  0.0004,  0.0075,  0.0109],
        [ 0.0207,  0.0207,  0.0766,  ..., -0.0105, -0.0137, -0.0025],
        [ 0.0704,  0.0215, -0.0091,  ..., -0.0205,  0.0170,  0.0243]],
       device='cuda:0', grad_fn=<ViewBackward0>)


In [57]:
def AA(Anderson_map,s):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    length = len(Anderson_map)

    theta = torch.zeros(length-1,device=device, requires_grad=True)
    optimizer = torch.optim.Adam([theta], lr=0.01)

    for step in range(100):
        optimizer.zero_grad()
        sum_all = torch.zeros(s,requires_grad=True,device=device)
        
        for i in range(length-1):
            _,F=Anderson_map[i]
            _,F1=Anderson_map[i+1]
            sum_all = sum_all + theta[i]*(F1-F)
        
        _,F = Anderson_map[length-1]
        loss = torch.norm(F - sum_all,p='fro')
        loss.backward()
        optimizer.step()
    
    ans,_ = Anderson_map[length-1]
    for i in range(length-1):
        G,_=Anderson_map[i]
        G1,_=Anderson_map[i+1]
        ans = ans - theta[i]*(G1-G)
    
    return ans.detach()
    



In [58]:
#ADMM Frobenius Normalize - Anderson Accelerate
import torch

Anderson_map = []
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

layer = alexnet_model.features[3].weight.to(device)
#layer = layer.to(device)
s = layer.shape
k = s[3]
n = 13

F = createFourierMatrix(k,n)

# Constants (example values, replace these with actual data)
all_one = torch.ones(n*n,device=device)
H0 = torch.reshape(layer,(s[0]*s[1],s[2]*s[3])).to(device)
print(H0.shape)
F = F.to(device)
print(F.shape)
F_real, F_imag = torch.real(F), torch.imag(F)

# Initialize optimization variables (H and lambda)
H = torch.rand_like(H0,dtype = torch.float32).to(device)
U = torch.zeros((s[0]*s[1],n*n),dtype=torch.complex64).to(device)
Hf = torch.zeros((s[0]*s[1],n*n),dtype=torch.complex64).to(device)

H_def = H
U_def = U
Hf_def = Hf

res_prev = 1000000
reset=True
rho = 0.1
m = 3
iter = 0
#i = 0
#optimizer = torch.optim.Adam([H], lr=0.01)
#while pri_res>0.5:
for i in range(10):
    # Optimize H
    H = SGDminimize(H0,Hf,U,F,rho)

    H_fourier = torch.zeros(Hf.shape,dtype = torch.complex64).to(device)
    H_fourier.real = H@torch.real(F.T)
    H_fourier.imag = H@torch.imag(F.T)
    #print(H_fourier)
    #Optimize Hf

    Hf_0 = H_fourier + U
    Y1 = torch.zeros(Hf_0.shape,dtype=torch.complex64,device=device)
    Y2 = torch.zeros(Hf_0.shape,dtype=torch.complex64,device=device)
    
    Hf_1 = Hf_0
    Hf_2 = Hf_0
    Hf_z = Hf_0
    H_bar = (Hf_1+Hf_2)/2
    Y_bar = (Y1 + Y2)/2
    
    for j in range(100):
        X1 = Hf_z - Y1
        X1_mat = torch.permute(torch.reshape(X1,(s[0],s[1],n,n)),(2,3,0,1))
        X1_norm = torch.clamp(torch.sum(torch.abs(X1_mat), dim=2),min=1.0)
        Hf_1 = torch.div(X1_mat,torch.reshape(X1_norm,((n,n,1,s[1]))))
        Hf_1 = torch.reshape(torch.permute(Hf_1,(2,3,0,1)),(s[0]*s[1],n*n))

        X1 = Hf_z - Y2
        X1_mat = torch.permute(torch.reshape(X1,(s[0],s[1],n,n)),(2,3,0,1))
        X1_norm = torch.clamp(torch.sum(torch.abs(X1_mat), dim=3),min=1.0)
        Hf_2 = torch.div(X1_mat,torch.reshape(X1_norm,((n,n,s[0],1))))
        Hf_2 = torch.reshape(torch.permute(Hf_2,(2,3,0,1)),(s[0]*s[1],n*n))

        H_bar = (Hf_1+Hf_2)/2
        Hf_z = SGDMinProject(Hf_0,Y_bar,H_bar,0.01)
        
        Y1 = Y1 + (Hf_1 - Hf_z)
        Y2 = Y2 + (Hf_2 - Hf_z)
        Y_bar = (Y1 + Y2)/2
        if(j%10 == 0):
            print(f'----------------Step {j}: Primal Residual:{torch.norm(Hf_z - H_bar,p='fro')}')

    dual_res = torch.norm(Hf-Hf_z,p='fro')**2    
    Hf = Hf_z

    # Update U
    U = U + H_fourier - Hf 
    pri_res = torch.norm(H_fourier - Hf, p='fro')**2
    #i=i+1

    res = pri_res+dual_res
    if((reset==True) or (res<res_prev)):
        H_def = H
        Hf_def = Hf
        U_def = U
        res_prev = res
        reset = False
        iter = iter+1
        Anderson_map.append((torch.hstack((Hf,U)),torch.hstack((Hf-Hf_z,H_fourier-Hf))))
        if(iter>(m+1)):
            Anderson_map.pop(0)

        G = AA(Anderson_map,(s[0]*s[1],n*n*2))
        Hf = G[:,n*n]
        U = G[:,n*n]
    else:
        Hf = Hf_def
        U = U_def
        reset = True
    
    if(i%10 == 0):
        print(f'Step {i}: Primal Residual:{pri_res}')
# Results
print(f'Optimized H:\n{H}')

torch.Size([12288, 25])
torch.Size([169, 25])
----------------Step 0: Primal Residual:21.07489013671875
----------------Step 10: Primal Residual:19.143449783325195
----------------Step 20: Primal Residual:17.391775131225586
----------------Step 30: Primal Residual:15.80850601196289
----------------Step 40: Primal Residual:14.377713203430176
----------------Step 50: Primal Residual:13.086310386657715
----------------Step 60: Primal Residual:11.919387817382812
----------------Step 70: Primal Residual:10.866209030151367
----------------Step 80: Primal Residual:9.915656089782715
----------------Step 90: Primal Residual:9.0564603805542


ValueError: too many values to unpack (expected 2)

In [10]:
#ADMM L1-LInfty - WORKING !!!!!!!!!!!!!!!!!!!!!!!!!!!!

import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

layer = alexnet_model.features[8].weight.to(device)
layer = layer.to(device)
s = layer.shape
k = s[3]
n = 13

F = createFourierMatrix(k,n)

# Constants (example values, replace these with actual data)
all_one = torch.ones(n*n,device=device)
H0 = torch.reshape(layer,(s[0]*s[1],s[2]*s[3])).to(device)
print(H0.shape)
F = F.to(device)
print(F.shape)
F_real, F_imag = torch.real(F), torch.imag(F)

# Initialize optimization variables (H and lambda)
H = torch.rand_like(H0,dtype = torch.float32).to(device)
U = torch.zeros((s[0]*s[1],n*n),dtype=torch.complex64).to(device)
Hf = torch.zeros((s[0]*s[1],n*n),dtype=torch.complex64).to(device)
Hf1 = torch.zeros((s[0]*s[1],n*n),dtype=torch.complex64).to(device)
Hf2 = torch.zeros((s[0]*s[1],n*n),dtype=torch.complex64).to(device)
U1 = torch.zeros((s[0]*s[1],n*n),dtype=torch.complex64).to(device)
U2 = torch.zeros((s[0]*s[1],n*n),dtype=torch.complex64).to(device)

H_bar = torch.zeros((s[0]*s[1],n*n),dtype=torch.complex64).to(device)
U_bar = torch.zeros((s[0]*s[1],n*n),dtype=torch.complex64).to(device)

pri_res = 100
rho = 0.1

#i = 0
#optimizer = torch.optim.Adam([H], lr=0.01)
#while pri_res>0.5:
for i in range(100):
    # Optimize H
    H = SGDminimize(H0,Hf,U,F,rho)

    X1 =  Hf - U1
    X1_mat = torch.permute(torch.reshape(X1,(s[0],s[1],n,n)),(2,3,0,1))
    X1_norm = torch.clamp(torch.sum(torch.abs(X1_mat), dim=2),min=1.0)
    Hf1 = torch.div(X1_mat,torch.reshape(X1_norm,((n,n,1,s[1]))))
    Hf1 = torch.reshape(torch.permute(Hf1,(2,3,0,1)),(s[0]*s[1],n*n))

    X1 = Hf - U2
    X1_mat = torch.permute(torch.reshape(X1,(s[0],s[1],n,n)),(2,3,0,1))
    X1_norm = torch.clamp(torch.sum(torch.abs(X1_mat), dim=3),min=1.0)
    Hf2 = torch.div(X1_mat,torch.reshape(X1_norm,((n,n,s[0],1))))
    Hf2 = torch.reshape(torch.permute(Hf2,(2,3,0,1)),(s[0]*s[1],n*n))

    H_fourier = torch.zeros(Hf.shape,dtype = torch.complex64).to(device)
    H_fourier.real = H@torch.real(F.T)
    H_fourier.imag = H@torch.imag(F.T)
    H_bar = (H_fourier + Hf1 + Hf2)/3
    Hf = H_bar + U_bar
    
    U = U + H_fourier - Hf
    U1 = U1 + Hf1 - Hf
    U2 = U2 + Hf2 - Hf
    U_bar = (U1+U2+U)/3
    pri_res = torch.norm(H_fourier - Hf, p='fro')
    #i=i+1
    
    if(i%10 == 0):
        print(f'Step {i}: Primal Residual:{pri_res}')
# Results
print(f'Optimized H:\n{H}')

torch.Size([98304, 9])
torch.Size([169, 9])
Step 0: Primal Residual:23.97040557861328
Step 10: Primal Residual:0.5088467597961426
Step 20: Primal Residual:0.21946126222610474
Step 30: Primal Residual:0.13900388777256012
Step 40: Primal Residual:0.10494109243154526
Step 50: Primal Residual:0.08867479860782623
Step 60: Primal Residual:0.08029833436012268
Step 70: Primal Residual:0.07572028785943985
Step 80: Primal Residual:0.07296615093946457
Step 90: Primal Residual:0.07111893594264984
Optimized H:
tensor([[ 3.0407e-04, -2.0102e-04, -5.6735e-04,  ..., -2.1678e-03,
          8.2489e-04, -1.0365e-03],
        [ 1.5948e-03, -1.5966e-04,  1.1119e-03,  ...,  1.1945e-04,
          5.3272e-04,  6.6553e-04],
        [-1.8734e-03, -1.2673e-03,  9.8569e-04,  ..., -1.5655e-03,
         -1.7978e-03,  1.9610e-03],
        ...,
        [-7.5574e-05,  1.4317e-03,  1.8691e-03,  ...,  3.2356e-04,
          1.6334e-03, -4.0907e-04],
        [-1.4631e-03, -5.4712e-04,  1.3728e-03,  ..., -1.4042e-03,
     

In [11]:
#Output Test
layer_wt = torch.reshape(H,(s[0],s[1],s[2],s[3]))
print(f'Lipschitz Constant: {computeLayerLipschitzFourier(layer_wt,n)}')
print(f'Difference: {torch.norm((H - H0), p="fro")}')
print(f'Original H:\n{H0}')

Lipschitz Constant: 0.3220367133617401
Difference: 25.249536514282227
Original H:
tensor([[-0.0020, -0.0081, -0.0114,  ..., -0.0541, -0.0012, -0.0244],
        [ 0.0350,  0.0133,  0.0260,  ...,  0.0035,  0.0181,  0.0147],
        [-0.0572, -0.0474,  0.0019,  ..., -0.0515, -0.0490,  0.0254],
        ...,
        [ 0.0133,  0.0259,  0.0499,  ...,  0.0153,  0.0184,  0.0001],
        [-0.0324, -0.0163,  0.0237,  ..., -0.0236,  0.0041,  0.0363],
        [-0.0293, -0.0176, -0.0268,  ...,  0.0003,  0.0701,  0.0068]],
       device='cuda:0', grad_fn=<ViewBackward0>)


In [1]:
import cvxpy as cp

In [ ]:
#ADMM L1-LInfty - CVXPY

import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

layer = alexnet_model.features[3].weight.to(device)
layer = layer.to(device)
s = layer.shape
k = s[3]
n = 13

F = createFourierMatrix(k,n)

# Constants (example values, replace these with actual data)
all_one = torch.ones(n*n,device=device)
H0 = torch.reshape(layer,(s[0]*s[1],s[2]*s[3])).to(device)
print(H0.shape)
F = F.to(device)
print(F.shape)
F_real, F_imag = torch.real(F), torch.imag(F)

# Initialize optimization variables (H and lambda)
H = torch.rand_like(H0,dtype = torch.float32).to(device)
U = torch.zeros((s[0]*s[1],n*n),dtype=torch.complex64).to(device)
Hf = torch.zeros((s[0]*s[1],n*n),dtype=torch.complex64).to(device)
Hf1 = torch.zeros((s[0]*s[1],n*n),dtype=torch.complex64).to(device)
Hf2 = torch.zeros((s[0]*s[1],n*n),dtype=torch.complex64).to(device)
U1 = torch.zeros((s[0]*s[1],n*n),dtype=torch.complex64).to(device)
U2 = torch.zeros((s[0]*s[1],n*n),dtype=torch.complex64).to(device)

H_bar = torch.zeros((s[0]*s[1],n*n),dtype=torch.complex64).to(device)
U_bar = torch.zeros((s[0]*s[1],n*n),dtype=torch.complex64).to(device)

pri_res = 100
rho = 0.1

#i = 0
#optimizer = torch.optim.Adam([H], lr=0.01)
#while pri_res>0.5:
for i in range(100):
    # Optimize H
    H = SGDminimize(H0,Hf,U,F,rho)

    X1 =  Hf - U1
    X2 = Hf - U2
    for j in range(n*n):
        c1 = torch.reshape(X1[:,j],(s[0],s[1]))
        c1_proj = cp.Variable((s[0],s[1]),complex=True)
        objective = cp.Minimize(cp.norm(c1_proj - (c1.cpu()).numpy(),'fro'))
        constraints = [cp.norm(c1_proj,1)<=1]
        prob = cp.Problem(objective,constraints)
        prob.solve(solver=cp.SCS)
        c1 = c1_proj.value()
        X1[:,j] = torch.reshape(torch.from_numpy(c1),s[0]*s[1])

        c2 = torch.reshape(X2[:,j],(s[0],s[1]))
        c2_proj = cp.Variable((s[0],s[1]),complex=True)
        objective = cp.Minimize(cp.norm(c2_proj - (c2.cpu()).numpy(),'fro'))
        constraints = [cp.norm(c2_proj,1)<=1]
        prob = cp.Problem(objective,constraints)
        prob.solve(solver=cp.SCS)
        c2 = c2_proj.value()
        X2[:,j] = torch.reshape(torch.from_numpy(c2),s[0]*s[1])

    Hf1 = X1
    Hf2 = X2
                        
    H_fourier = torch.zeros(Hf.shape,dtype = torch.complex64).to(device)
    H_fourier.real = H@torch.real(F.T)
    H_fourier.imag = H@torch.imag(F.T)
    H_bar = (H_fourier + Hf1 + Hf2)/3
    Hf = H_bar + U_bar
    
    U = U + H_fourier - Hf
    U1 = U1 + Hf1 - Hf
    U2 = U2 + Hf2 - Hf
    U_bar = (U1+U2+U)/3
    pri_res = torch.norm(H_fourier - Hf, p='fro')
    #i=i+1
    
    if(i%10 == 0):
        print(f'Step {i}: Primal Residual:{pri_res}')
# Results
print(f'Optimized H:\n{H}')

In [41]:
#ADMM L1-LInfty

import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#layer = alexnet_model.features[8].weight.to(device)
layer = layer.to(device)
s = layer.shape
k = s[3]
n = 40

F = createFourierMatrix(k,n)

# Constants (example values, replace these with actual data)
all_one = torch.ones(n*n,device=device)
H0 = torch.reshape(layer,(s[0]*s[1],s[2]*s[3])).to(device)
print(H0.shape)
F = F.to(device)
print(F.shape)
F_real, F_imag = torch.real(F), torch.imag(F)

# Initialize optimization variables (H and lambda)
H = torch.rand_like(H0,dtype = torch.float32).to(device)
U = torch.zeros((s[0]*s[1],n*n),dtype=torch.complex64).to(device)
Hf = torch.zeros((s[0]*s[1],n*n),dtype=torch.complex64).to(device)
Hf1 = torch.zeros((s[0]*s[1],n*n),dtype=torch.complex64).to(device)
Hf2 = torch.zeros((s[0]*s[1],n*n),dtype=torch.complex64).to(device)
U1 = torch.zeros((s[0]*s[1],n*n),dtype=torch.complex64).to(device)
U2 = torch.zeros((s[0]*s[1],n*n),dtype=torch.complex64).to(device)

H_bar = torch.zeros((s[0]*s[1],n*n),dtype=torch.complex64).to(device)
U_bar = torch.zeros((s[0]*s[1],n*n),dtype=torch.complex64).to(device)

res = 100
rho = 0.1

i = 0
#optimizer = torch.optim.Adam([H], lr=0.01)
#while(res>0.05)
#while pri_res>0.01:
for i in range(200):
    # Optimize H
    H = SGDminimize(H0,Hf,U,F,rho)

    X1 =  Hf - U1
    X1_mat = torch.permute(torch.reshape(X1,(s[0],s[1],n,n)),(2,3,0,1))
    X1_norm = torch.clamp(torch.sum(torch.abs(X1_mat), dim=2),min=1.0)
    Hf1 = torch.div(X1_mat,torch.reshape(X1_norm,((n,n,1,s[1]))))
    Hf1 = torch.reshape(torch.permute(Hf1,(2,3,0,1)),(s[0]*s[1],n*n))

    X1 = Hf - U2
    X1_mat = torch.permute(torch.reshape(X1,(s[0],s[1],n,n)),(2,3,0,1))
    X1_norm = torch.clamp(torch.sum(torch.abs(X1_mat), dim=3),min=1.0)
    Hf2 = torch.div(X1_mat,torch.reshape(X1_norm,((n,n,s[0],1))))
    Hf2 = torch.reshape(torch.permute(Hf2,(2,3,0,1)),(s[0]*s[1],n*n))

    H_fourier = torch.zeros(Hf.shape,dtype = torch.complex64).to(device)
    H_fourier.real = H@torch.real(F.T)
    H_fourier.imag = H@torch.imag(F.T)
    dual_res = torch.norm(H_bar - (H_fourier + Hf1 + Hf2)/3,p='fro')**2
    H_bar = (H_fourier + Hf1 + Hf2)/3
    Hf = H_bar + U_bar

    U = U + H_fourier - Hf
    U1 = U1 + Hf1 - Hf
    U2 = U2 + Hf2 - Hf
    U_bar = (U1+U2+U)/3

    pri_res = torch.norm(H_fourier - H_bar,p='fro')**2 + torch.norm(Hf1 - H_bar,p='fro')**2 + torch.norm(Hf2 - H_bar,p='fro')**2
    #pri_res = torch.norm(H_fourier - Hf, p='fro')

    if(i%10 == 0):
        print(f'Step {i}: Primal Residual:{pri_res},Dual Residual:{dual_res}')
    
    #i=i+1
    
# Results
print(f'Optimized H:\n{H}') 
layer_wt = torch.reshape(H,(s[0],s[1],s[2],s[3]))
print(f'Lipschitz Constant: {computeLayerLipschitzFourier(layer_wt,n)}')
print(f'Difference: {torch.norm((H - H0), p="fro")**2}')

torch.Size([36, 9])
torch.Size([1600, 9])
Step 0: Primal Residual:16.55470085144043,Dual Residual:2.7591171264648438
Step 10: Primal Residual:0.7833991050720215,Dual Residual:1.9197267293930054
Step 20: Primal Residual:0.6151298880577087,Dual Residual:0.7685045003890991
Step 30: Primal Residual:0.08681049942970276,Dual Residual:0.49097153544425964
Step 40: Primal Residual:0.08625029027462006,Dual Residual:0.39104995131492615
Step 50: Primal Residual:0.3090822398662567,Dual Residual:0.1815582811832428
Step 60: Primal Residual:0.16735610365867615,Dual Residual:0.1204300969839096
Step 70: Primal Residual:0.11745654046535492,Dual Residual:0.06231284886598587
Step 80: Primal Residual:0.06214126944541931,Dual Residual:0.043409936130046844
Step 90: Primal Residual:0.04422123730182648,Dual Residual:0.02841004729270935
Step 100: Primal Residual:0.030799565836787224,Dual Residual:0.019342172890901566
Step 110: Primal Residual:0.023048851639032364,Dual Residual:0.012206587009131908
Step 120: Prim

In [30]:
layer_wt = torch.reshape(H,(s[0],s[1],s[2],s[3]))
print(f'Lipschitz Constant: {computeLayerLipschitzFourier(layer_wt,n)}')

Lipschitz Constant: 0.995888888835907


In [36]:
mat = np.random.rand(3,5)
print(mat)

x = cp.Variable((3,5))
objective=cp.Minimize(cp.norm(x-mat,"fro"))
constraints = [cp.norm(x,1)<=1]

cp.Problem(objective,constraints).solve();
print(x.value)

y = np.sum(mat,axis=0)
np.divide()

[[0.77319067 0.72823359 0.91769675 0.30920708 0.20493604]
 [0.56429897 0.58296617 0.52434477 0.52120644 0.80432528]
 [0.21412907 0.2877764  0.75531956 0.01114734 0.52108853]]
[[0.58932538 0.5285762  0.5185777  0.3092322  0.02813078]
 [0.38043675 0.38331079 0.1252202  0.52122915 0.62755118]
 [0.03023787 0.08811301 0.35620212 0.0111572  0.34431805]]


In [19]:
#ADMM Frobenius Normalize
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#layer = alexnet_model.features[3].weight.to(device)
#layer = layer.to(device)
s = layer.shape
k = s[3]
n = 40

F = createFourierMatrix(k,n)

# Constants (example values, replace these with actual data)
all_one = torch.ones(n*n,device=device)
H0 = torch.reshape(layer,(s[0]*s[1],s[2]*s[3])).to(device)
print(H0.shape)
F = F.to(device)
print(F.shape)
F_real, F_imag = torch.real(F), torch.imag(F)

# Initialize optimization variables (H and lambda)
H = torch.rand_like(H0,dtype = torch.float32).to(device)
print(H.shape)
U = torch.zeros((s[0]*s[1],n*n),dtype=torch.complex64).to(device)
Hf = torch.zeros((s[0]*s[1],n*n),dtype=torch.complex64).to(device)

pri_res = 100
rho = 0.1

#i = 0
#while pri_res>0.5:
for i in range(10):
    # Optimize H
    H = SGDminimize(H0,Hf,U,F,rho)
    print(H)
    H_fourier = torch.zeros(Hf.shape,dtype = torch.complex64).to(device)
    H_fourier.real = H@torch.real(F.T)
    H_fourier.imag = H@torch.imag(F.T)

    #Optimize Hf

    Hf = H_fourier + U
    H_frob = torch.clamp(torch.sqrt(torch.sum(torch.square(torch.abs(Hf)),dim = 0)),min=1)
    s_f = H_frob.shape
    Hf = torch.div(Hf,torch.reshape(H_frob,(1,s_f[0])))

    # Update U
    U = U + H_fourier - Hf
    pri_res = torch.norm(H_fourier - Hf, p='fro')
    #i=i+1
    if(i%10 == 0):
        print(f'Step {i}: Primal Residual:{pri_res}')
# Results
print(f'Optimized H:\n{H}')

torch.Size([12288, 25])
torch.Size([1600, 25])
torch.Size([12288, 25])
tensor([[ 4.4748e-05,  1.7698e-05,  4.5946e-04,  ...,  3.8863e-04,
          2.0615e-04, -5.8186e-05],
        [-6.7374e-05, -2.3564e-04, -4.0030e-04,  ..., -2.7144e-05,
         -2.0457e-04,  6.9573e-05],
        [-1.5539e-04, -7.3378e-04, -1.1887e-03,  ..., -9.9195e-05,
         -1.0101e-03,  1.7723e-04],
        ...,
        [-7.9884e-05, -4.0650e-04, -1.9423e-04,  ...,  5.4587e-06,
          9.2837e-05,  1.3481e-04],
        [ 2.5528e-04,  2.5600e-04,  9.4541e-04,  ..., -1.3024e-04,
         -1.6855e-04, -3.0799e-05],
        [ 8.6946e-04,  2.6536e-04, -1.1270e-04,  ..., -2.5305e-04,
          2.1015e-04,  3.0038e-04]], device='cuda:0')
Step 0: Primal Residual:0.0
tensor([[ 8.8943e-05,  3.5178e-05,  9.1326e-04,  ...,  7.7247e-04,
          4.0975e-04, -1.1565e-04],
        [-1.3392e-04, -4.6836e-04, -7.9565e-04,  ..., -5.3954e-05,
         -4.0661e-04,  1.3829e-04],
        [-3.0886e-04, -1.4585e-03, -2.3626e-03

In [11]:
#Output Test
layer_wt = torch.reshape(H,(s[0],s[1],s[2],s[3]))
print(f'Lipschitz Constant: {computeLayerLipschitzFourier(layer_wt,n)}')
print(f'Difference:{torch.norm(H - H0, p="fro")**2}')
print(f'Original H:\n{H0}')

Lipschitz Constant: 0.583524763584137
Difference:637.15673828125
Original H:
tensor([[ 0.0036,  0.0014,  0.0372,  ...,  0.0315,  0.0167, -0.0047],
        [-0.0055, -0.0191, -0.0324,  ..., -0.0022, -0.0166,  0.0056],
        [-0.0126, -0.0594, -0.0963,  ..., -0.0080, -0.0818,  0.0144],
        ...,
        [-0.0065, -0.0329, -0.0157,  ...,  0.0004,  0.0075,  0.0109],
        [ 0.0207,  0.0207,  0.0766,  ..., -0.0105, -0.0137, -0.0025],
        [ 0.0704,  0.0215, -0.0091,  ..., -0.0205,  0.0170,  0.0243]],
       device='cuda:0', grad_fn=<ViewBackward0>)


In [57]:
import gc
torch.cuda.empty_cache()
gc.collect()

9353

In [50]:
#ADMM Frobenius Normalize - Heuristc
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

layer = alexnet_model.features[3].weight.to(device)
#layer = layer.to(device)
s = layer.shape
k = s[3]
n = 40

F = createFourierMatrix(n,n)

# Constants (example values, replace these with actual data)
H0 = zeroPad2DMatrix(layer,n).detach()
H0 = torch.reshape(H0,(s[0]*s[1],n*n)).to(device)

print(H0.shape)
#F = F.to(device)
#print(F.shape)
#F_real, F_imag = torch.real(F), torch.imag(F)

# Initialize optimization variables (H and lambda)
H = torch.rand_like(H0,dtype = torch.float32).to(device)
print(H.shape)
U = torch.zeros((s[0]*s[1],n*n),dtype=torch.complex64).to(device)
Hf = torch.zeros((s[0]*s[1],n*n),dtype=torch.complex64).to(device)

pri_res = 100
rho = 0.1

temp = 2*torch.eye(n*n).to(device) + rho*torch.real(F.H@F)
temp = temp.to('cpu')
inv_mat = torch.linalg.inv(temp)
inv_mat = inv_mat.to(device)

#i = 0
#optimizer = torch.optim.Adam([H], lr=0.01)
#while pri_res>0.5:
for i in range(100):
    # Optimize H

    x = (Hf-U)@torch.conj(F)
    H = (2*H0 + (rho) * torch.real(x))@(inv_mat)
    H = torch.reshape(H,(s[0],s[1],n,n))
    H = deZeroPad2DMatrix(H,k)
    H = zeroPad2DMatrix(H,n)
    H = torch.reshape(H,(s[0]*s[1],n*n))

    H_fourier = torch.zeros(Hf.shape,dtype = torch.complex64).to(device)
    H_fourier.real = H@torch.real(F.T)
    H_fourier.imag = H@torch.imag(F.T)

    dual_res = Hf
    Hf = H_fourier + U
    H_frob = torch.clamp(torch.sqrt(torch.sum(torch.square(torch.abs(Hf)),dim = 0)),min=1)
    #H_frob = torch.sum(torch.abs(Hf),dim = 0)
    #print(H_frob)
    s_f = H_frob.shape
    Hf = torch.div(Hf,torch.reshape(H_frob,(1,s_f[0])))
    dual_res = torch.norm(Hf - dual_res, p='fro')

    # Update U
    U = U + H_fourier - Hf
    pri_res = torch.norm(H_fourier - Hf, p='fro')
    #i=i+1
    if(i%10 == 0):
        print(f'Step {i}: Primal Residual:{pri_res} --------- Dual Residual:{dual_res}')
    
    del x,H_fourier,s_f
# Results
print(f'Optimized H:\n{H}')

torch.Size([12288, 1600])
torch.Size([12288, 1600])
Step 0: Primal Residual:0.0 --------- Dual Residual:12.910954475402832
Step 10: Primal Residual:0.5220763683319092 --------- Dual Residual:0.9866501092910767
Step 20: Primal Residual:0.054465826600790024 --------- Dual Residual:0.09499482065439224
Step 30: Primal Residual:0.011437932029366493 --------- Dual Residual:0.015981443226337433
Step 40: Primal Residual:0.003817094024270773 --------- Dual Residual:0.003910079598426819
Step 50: Primal Residual:0.0012178265023976564 --------- Dual Residual:0.001061644172295928
Step 60: Primal Residual:0.0004802555777132511 --------- Dual Residual:0.00035684873000718653
Step 70: Primal Residual:0.00022325586178340018 --------- Dual Residual:0.00016058441542554647
Step 80: Primal Residual:0.00010772810492198914 --------- Dual Residual:7.935526082292199e-05
Step 90: Primal Residual:5.301719647832215e-05 --------- Dual Residual:3.978165841544978e-05
Optimized H:
tensor([[ 3.3273e-03, -4.0544e-03,  2

In [51]:
#Output Test
A = torch.reshape(H,(s[0],s[1],n,n))
A = deZeroPad2DMatrix(A,k)
layer_wt = torch.reshape(A,(s[0],s[1],s[2],s[3]))
print(f'Lipschitz Constant: {computeLayerLipschitzFourier(layer_wt,n)}')
print(f'Difference:{torch.norm(H - H0, p="fro")**2}')
print(f'Original H:\n{H0}')

Lipschitz Constant: 0.5835258960723877
Difference:637.15673828125
Original H:
tensor([[ 0.0216, -0.0695,  0.0316,  ...,  0.0000,  0.0241, -0.0121],
        [ 0.0613,  0.0133,  0.0262,  ...,  0.0000, -0.0075,  0.0309],
        [ 0.2158,  0.0295, -0.0589,  ...,  0.0000, -0.0150, -0.1121],
        ...,
        [ 0.0448,  0.0221,  0.0354,  ...,  0.0000, -0.0541, -0.0547],
        [ 0.1288,  0.0619,  0.0294,  ...,  0.0000, -0.0010,  0.0554],
        [-0.0856, -0.0549, -0.0188,  ...,  0.0000,  0.0309, -0.0101]],
       device='cuda:0')


In [66]:
#ADMM Frobenius Normalize - Exact
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

layer = alexnet_model.features[3].weight.to(device)
layer = layer.to(device)
s = layer.shape
k = s[3]
n = 40

F = createFourierMatrix(k,n)

# Constants (example values, replace these with actual data)

H0 = torch.reshape(layer.detach(),(s[0]*s[1],k*k)).to(device)

print(H0.shape)
F = F.to(device)
#print(F.shape)
F_real, F_imag = torch.real(F), torch.imag(F)

# Initialize optimization variables (H and lambda)
H = torch.rand_like(H0,requires_grad=False,dtype = torch.float32).to(device)
print(H.shape)
U = torch.zeros((s[0]*s[1],n*n),requires_grad=False,dtype=torch.complex64).to(device)
Hf = torch.zeros((s[0]*s[1],n*n),requires_grad=False,dtype=torch.complex64).to(device)

pri_res = 100
rho = 0.1

temp = 2*torch.eye(k*k).to(device) + rho*torch.real(F.H@F)
temp = temp.to('cpu')
inv_mat = torch.linalg.inv(temp)
inv_mat = inv_mat.to(device)

#i = 0
#optimizer = torch.optim.Adam([H], lr=0.01)
#while pri_res>0.5:
for i in range(100):
    # Optimize H
    x = Hf - U
    H = (2*H0 + rho*torch.real(torch.conj(x)@F))@inv_mat
    #H = H.to(torch.float32)

    H_fourier = torch.zeros(Hf.shape,dtype = torch.complex64).to(device)
    H_fourier.real = H@torch.real(F.T)
    H_fourier.imag = H@torch.imag(F.T)
    #print(H_fourier)
    #Optimize Hf

    dual_res = Hf
    Hf = H_fourier + U
    H_frob = torch.clamp(torch.sqrt(torch.sum(torch.square(torch.abs(Hf)),dim = 0)),min=1)
    #H_frob = torch.sum(torch.abs(Hf),dim = 0)
    #print(H_frob)
    s_f = H_frob.shape
    Hf = torch.div(Hf,torch.reshape(H_frob,(1,s_f[0])))
    dual_res = torch.norm(Hf - dual_res, p='fro')

    # Update U
    U = U + H_fourier - Hf
    pri_res = torch.norm(H_fourier - Hf, p='fro')

    #i=i+1
    if(i%10 == 0):
        print(f'Step {i}: Primal Residual:{pri_res} --- Dual Residual:{dual_res}')
# Results
print(f'Optimized H:\n{H}')

torch.Size([12288, 25])
torch.Size([12288, 25])
Step 0: Primal Residual:0.0 --- Dual Residual:12.910954475402832
Step 10: Primal Residual:0.5220803022384644 --- Dual Residual:0.9866498112678528
Step 20: Primal Residual:0.05446581169962883 --- Dual Residual:0.094994455575943
Step 30: Primal Residual:0.011437954381108284 --- Dual Residual:0.015981435775756836
Step 40: Primal Residual:0.003816974116489291 --- Dual Residual:0.003910013474524021
Step 50: Primal Residual:0.001217905431985855 --- Dual Residual:0.001061499584466219
Step 60: Primal Residual:0.0004802561306860298 --- Dual Residual:0.00035629564081318676
Step 70: Primal Residual:0.00022331906075123698 --- Dual Residual:0.00015967278159223497
Step 80: Primal Residual:0.00010823555203387514 --- Dual Residual:7.790025847498327e-05
Step 90: Primal Residual:5.3875643061473966e-05 --- Dual Residual:3.769535032915883e-05
Optimized H:
tensor([[-1.3102e-04, -5.5373e-05,  1.0151e-03,  ...,  1.9139e-03,
          5.0321e-05, -1.3983e-04],
 

In [ ]:
#Output Test
layer_wt = torch.reshape(H,(s[0],s[1],s[2],s[3]))
print(f'Lipschitz Constant: {computeLayerLipschitzFourier(layer_wt,n)}')
print(f'Difference:{torch.norm(H - H0, p="fro")**2}')
print(f'Original H:\n{H0}')

In [76]:
#ADMM Frobenius Normalize - Heuristc
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

layer = alexnet_model.features[3].weight.to(device)
#layer = layer.to(device)
s = layer.shape
k = s[3]
n = 40

F = createFourierMatrix(n,n)

# Constants (example values, replace these with actual data)
H0 = zeroPad2DMatrix(layer,n).detach()
H0 = torch.reshape(H0,(s[0]*s[1],n*n)).to(device)

print(H0.shape)
#F = F.to(device)
#print(F.shape)
#F_real, F_imag = torch.real(F), torch.imag(F)

# Initialize optimization variables (H and lambda)
H = torch.rand_like(H0,dtype = torch.float32).to(device)
print(H.shape)
U = torch.zeros((s[0]*s[1],n*n),dtype=torch.complex64).to(device)
Hf = torch.zeros((s[0]*s[1],n*n),dtype=torch.complex64).to(device)

pri_res = 100
rho = 0.1

temp = 2*torch.eye(n*n).to(device) + rho*torch.real(F.H@F)
temp = temp.to('cpu')
inv_mat = torch.linalg.inv(temp)
inv_mat = inv_mat.to(device)
print(torch.real(F.H@F))
#i = 0
#optimizer = torch.optim.Adam([H], lr=0.01)
#while pri_res>0.5:
for i in range(100):
    # Optimize H

    x = (Hf-U)@torch.conj(F)
    #H = (2*H0 + (rho) * torch.real(x))@(inv_mat)
    H = (2*H0 + (rho) * torch.real(x))/(2+rho*n*n)
    H = torch.reshape(H,(s[0],s[1],n,n))
    H = deZeroPad2DMatrix(H,k)
    H = zeroPad2DMatrix(H,n)
    H = torch.reshape(H,(s[0]*s[1],n*n))

    H_fourier = torch.zeros(Hf.shape,dtype = torch.complex64).to(device)
    H_fourier.real = H@torch.real(F.T)
    H_fourier.imag = H@torch.imag(F.T)

    dual_res = Hf
    Hf = H_fourier + U
    H_frob = torch.clamp(torch.sqrt(torch.sum(torch.square(torch.abs(Hf)),dim = 0)),min=1)
    #H_frob = torch.sum(torch.abs(Hf),dim = 0)
    #print(H_frob)
    s_f = H_frob.shape
    Hf = torch.div(Hf,torch.reshape(H_frob,(1,s_f[0])))
    dual_res = torch.norm(Hf - dual_res, p='fro')

    # Update U
    U = U + H_fourier - Hf
    pri_res = torch.norm(H_fourier - Hf, p='fro')
    #i=i+1
    if(i%10 == 0):
        print(f'Step {i}: Primal Residual:{pri_res} --------- Dual Residual:{dual_res}')
    
    del x,H_fourier,s_f
# Results
print(f'Optimized H:\n{H}')

torch.Size([12288, 1600])
torch.Size([12288, 1600])
tensor([[ 1.6000e+03, -3.0037e-04,  1.4882e-04,  ..., -4.0298e-06,
         -9.7487e-06,  6.5536e-06],
        [-2.9931e-04,  1.6000e+03, -7.3511e-06,  ..., -1.2481e-06,
         -9.5871e-07,  2.7905e-06],
        [ 1.4882e-04, -6.6557e-06,  1.6000e+03,  ..., -2.4866e-06,
          4.8422e-06, -5.9385e-07],
        ...,
        [-6.2476e-06, -1.2481e-06, -3.3441e-07,  ...,  1.6000e+03,
          5.6859e-05,  2.6223e-05],
        [-9.7487e-06,  2.7772e-06,  4.8422e-06,  ...,  5.5696e-05,
          1.6000e+03,  2.0068e-04],
        [ 8.4459e-06,  2.7905e-06, -6.8718e-08,  ...,  2.6223e-05,
          2.0295e-04,  1.6000e+03]], device='cuda:0')
Step 0: Primal Residual:0.0 --------- Dual Residual:12.910962104797363
Step 10: Primal Residual:0.5220791697502136 --------- Dual Residual:0.9866526126861572
Step 20: Primal Residual:0.0544651560485363 --------- Dual Residual:0.09499283879995346
Step 30: Primal Residual:0.011437925510108471 -------

In [77]:
#Output Test
A = torch.reshape(H,(s[0],s[1],n,n))
A = deZeroPad2DMatrix(A,k)
layer_wt = torch.reshape(A,(s[0],s[1],s[2],s[3]))
print(f'Lipschitz Constant: {computeLayerLipschitzFourier(layer_wt,n)}')
print(f'Difference:{torch.norm(H - H0, p="fro")**2}')
print(f'Original H:\n{H0}')

Lipschitz Constant: 0.5835268497467041
Difference:637.15673828125
Original H:
tensor([[ 0.0216, -0.0695,  0.0316,  ...,  0.0000,  0.0241, -0.0121],
        [ 0.0613,  0.0133,  0.0262,  ...,  0.0000, -0.0075,  0.0309],
        [ 0.2158,  0.0295, -0.0589,  ...,  0.0000, -0.0150, -0.1121],
        ...,
        [ 0.0448,  0.0221,  0.0354,  ...,  0.0000, -0.0541, -0.0547],
        [ 0.1288,  0.0619,  0.0294,  ...,  0.0000, -0.0010,  0.0554],
        [-0.0856, -0.0549, -0.0188,  ...,  0.0000,  0.0309, -0.0101]],
       device='cuda:0')


In [67]:
import gc
torch.cuda.empty_cache()
gc.collect()

0

In [148]:
#ADMM Frobenius Normalize - Heuristc
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

layer = alexnet_model.features[3].weight.to(device)
#layer = layer.to(device)
s = layer.shape
k = s[3]
n = 40

#F = createFourierMatrix(k,n)

# Constants (example values, replace these with actual data)
H0 = zeroPad2DMatrix(layer,n).detach()
H0 = torch.reshape(H0,(s[0]*s[1],n,n)).to(device)

print(H0.shape)
#F = F.to(device)
#print(F.shape)
#F_real, F_imag = torch.real(F), torch.imag(F)

# Initialize optimization variables (H and lambda)
H = torch.rand_like(H0,dtype = torch.float32).to(device)
print(H.shape)
U = torch.zeros((s[0]*s[1],n,n),dtype=torch.complex64).to(device)
Hf = torch.zeros((s[0]*s[1],n,n),dtype=torch.complex64).to(device)

pri_res = 100
rho = 0.01

#i = 0
#optimizer = torch.optim.Adam([H], lr=0.01)
#while pri_res>0.5:
for i in range(1000):
    # Optimize H
    x = torch.fft.ifft2(Hf-U)
    H = (2*H0 + (rho) * torch.real(x)*n*n)/(2+rho*n*n)
    H = torch.reshape(H,(s[0],s[1],n,n))
    H = deZeroPad2DMatrix(H,k)
    H = zeroPad2DMatrix(H,n)
    H = torch.reshape(H,(s[0]*s[1],n,n))

    H_fourier = torch.fft.fft2(H)

    #Optimize Hf

    dual_res = Hf
    Hf = H_fourier + U
    H_frob = torch.clamp(torch.sqrt(torch.sum(torch.square(torch.abs(Hf)),dim = 0)),min=1)
    #H_frob = torch.sum(torch.abs(Hf),dim = 0)
    #print(H_frob)
    s_f = H_frob.shape
    Hf = torch.div(Hf,torch.reshape(H_frob,(1,n,n)))
    dual_res = torch.norm(torch.fft.ifft2(Hf - dual_res), p='fro')*rho
    

    # Update U
    U = U + H_fourier - Hf
    pri_res = torch.norm(H_fourier - Hf, p='fro')
    #i=i+1
    if(i%100 == 0):
        print(f'Step {i}: Residual:{pri_res+dual_res}')
    
    del x,H_fourier,s_f
# Results
print(f'Optimized H:\n{H}')

torch.Size([12288, 40, 40])
torch.Size([12288, 40, 40])
Step 0: Residual:80.98329162597656
Step 100: Residual:0.03360621631145477
Step 200: Residual:0.01159454695880413
Step 300: Residual:0.0059705800376832485
Step 400: Residual:0.0033911247737705708
Step 500: Residual:0.0019716101232916117
Step 600: Residual:0.0011520414846017957
Step 700: Residual:0.0006738461670465767
Step 800: Residual:0.00039428286254405975
Step 900: Residual:0.00023074196360539645
Optimized H:
tensor([[[ 3.3273e-03, -4.0544e-03,  2.3649e-03,  ...,  0.0000e+00,
           2.1686e-03, -5.9101e-03],
         [-3.7785e-03,  2.9641e-03, -1.0997e-03,  ...,  0.0000e+00,
          -7.4911e-04,  6.0331e-04],
         [ 1.9139e-03,  5.0315e-05, -1.3983e-04,  ...,  0.0000e+00,
          -5.2857e-04,  1.3500e-04],
         ...,
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 1.0151e-03, -3.3122e-04, -3.8046e-05,  ...,  0.0000e+00,
          -1.3103e-04, -5.5

In [149]:
#Output Test
A = torch.reshape(H,(s[0],s[1],n,n))
A = deZeroPad2DMatrix(A,k)
layer_wt = torch.reshape(A,(s[0],s[1],s[2],s[3]))
print(f'Lipschitz Constant: {computeLayerLipschitzFourier(layer_wt,n)}')
print(f'Difference:{torch.norm(H - H0, p="fro")**2}')
print(f'Original H:\n{H0}')

Lipschitz Constant: 0.583531379699707
Difference:637.15673828125
Original H:
tensor([[[ 0.0216, -0.0695,  0.0316,  ...,  0.0000,  0.0210, -0.1062],
         [-0.0466,  0.0221, -0.0013,  ...,  0.0000, -0.0083, -0.0390],
         [ 0.0315,  0.0167, -0.0047,  ...,  0.0000, -0.0154,  0.0087],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0372, -0.0209,  0.0018,  ...,  0.0000,  0.0036,  0.0014],
         [ 0.0712, -0.0852,  0.0131,  ...,  0.0000,  0.0241, -0.0121]],

        [[ 0.0613,  0.0133,  0.0262,  ...,  0.0000,  0.0055,  0.0803],
         [ 0.0384, -0.0063,  0.0040,  ...,  0.0000, -0.0269,  0.0059],
         [-0.0022, -0.0166,  0.0056,  ...,  0.0000,  0.0021, -0.0363],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [-0.0324, -0.0220, -0.0121,  ...,  0.0000, -0.0055, -0.0191],
         [ 0.0319, -0.0076, -0.0147,  ...,  0.0000, -0.0075,  0.0309]],

        [[ 0.2158,  0.0295, -0.0589,  .